<a href="https://colab.research.google.com/github/camihu14520/Course-BDA/blob/main/11367008.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
df1 = pd.read_csv('/content/drive/My Drive/all-cancelled-flights.csv')
df2 = pd.read_csv('/content/drive/My Drive/cancellation-codes.csv')
import os
merged_data = pd.merge(df1, df2, left_on='CANCELLATION_CODE', right_on='Code', how='left')
print(merged_data.head())
output_path = '/content/drive/MyDrive/Task2/merged_cancelled_flights.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
merged_data.to_csv(output_path, index=False)
print(f"合併後的檔案: {output_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
  FLIGHT_DATE       DAY OP_CARRIER               CARRIER_NAME  \
0  2020-10-01  Thursday         BA   AnyCompany Blue Airlines   
1  2020-10-01  Thursday         BA   AnyCompany Blue Airlines   
2  2020-10-01  Thursday         GA  AnyCompany Green Airlines   
3  2020-10-01  Thursday         GA  AnyCompany Green Airlines   
4  2020-10-01  Thursday         GA  AnyCompany Green Airlines   

   OP_CARRIER_FL_NUM ORIGIN DISPLAY_AIRPORT_NAME_ORIGIN DEST  \
0               5029    ATL           Atlanta Municipal  MLB   
1               5069    MLB          Melbourne Regional  ATL   
2                134    DFW  Dallas Fort Worth Regional  OGG   
3               1960    OGG             Kahului Airport  PHX   
4               2242    LAX   Los Angeles International  KOA   

          DISPLAY_AIRPORT_NAME_DEST CANCELLATION_CODE  NUMBER_OF_FLIGHTS  \
0                Me

In [12]:
cancel_counts = df1['CANCELLATION_CODE'].value_counts()
print(cancel_counts)
cancel_percentage = cancel_counts / cancel_counts.sum() * 100
print(cancel_percentage)

CANCELLATION_CODE
B    133820
A     94701
C     22107
D      3233
Name: count, dtype: int64
CANCELLATION_CODE
B    52.713887
A    37.304273
C     8.708309
D     1.273532
Name: count, dtype: float64


In [22]:
flight_counts = df1['OP_CARRIER_FL_NUM'].value_counts()
most_frequent_flight = flight_counts.idxmax()
most_frequent_count = flight_counts.max()
total_flights = flight_counts.sum()
most_frequent_percentage = (most_frequent_count / total_flights) * 100
print(f"取消最多的航班班機號是: {most_frequent_flight}，取消了 {most_frequent_count} 次，占比為 {most_frequent_percentage:.2f}%")



取消最多的航班班機號是: 76，取消了 153 次，占比為 0.06%


In [14]:
df1['FLIGHT_DATE'] = pd.to_datetime(df1['FLIGHT_DATE'])
ogg_flights = df1[df1['ORIGIN'] == 'OGG']
df1_2020 = ogg_flights[(ogg_flights['FLIGHT_DATE'] >= '2020-10-01') & (ogg_flights['FLIGHT_DATE'] <= '2020-12-31')]
df1_2021 = ogg_flights[(ogg_flights['FLIGHT_DATE'] >= '2021-01-01') & (ogg_flights['FLIGHT_DATE'] <= '2021-12-31')]
df1_2022 = ogg_flights[(ogg_flights['FLIGHT_DATE'] >= '2022-01-01') & (ogg_flights['FLIGHT_DATE'] <= '2022-10-31')]
def calculate_cancellation_stats(dataframe, year):
    cancelled_flights = dataframe[dataframe['CANCELLATION_CODE'].notna()]
    cancelled_percentage = (len(cancelled_flights) / len(dataframe)) * 100 if len(dataframe) > 0 else 0
    cancel_reason_percentage = cancelled_flights['CANCELLATION_CODE'].value_counts(normalize=True) * 100
    print(f"{year} 年的取消原因占比:")
    print(cancel_reason_percentage)
calculate_cancellation_stats(df1_2020, 2020)
calculate_cancellation_stats(df1_2021, 2021)
calculate_cancellation_stats(df1_2022, 2022)

2020 年的取消原因占比:
CANCELLATION_CODE
D    87.958115
A    10.994764
B     1.047120
Name: proportion, dtype: float64
2021 年的取消原因占比:
CANCELLATION_CODE
A    91.071429
B     6.696429
D     1.785714
C     0.446429
Name: proportion, dtype: float64
2022 年的取消原因占比:
CANCELLATION_CODE
A    97.340426
B     2.127660
C     0.531915
Name: proportion, dtype: float64


In [15]:
iad_flights = df1[df1['ORIGIN'] == 'IAD']
most_frequent_carrier = iad_flights['OP_CARRIER'].value_counts().idxmax()
most_frequent_carrier_count = iad_flights['OP_CARRIER'].value_counts().max()
carrier_flights = iad_flights[iad_flights['OP_CARRIER'] == most_frequent_carrier]
cancelled_carrier_flights = carrier_flights[carrier_flights['CANCELLATION_CODE'].notna()]
most_frequent_cancellation = cancelled_carrier_flights['CANCELLATION_CODE'].value_counts().idxmax()
most_frequent_cancellation_count = cancelled_carrier_flights['CANCELLATION_CODE'].value_counts().max()
print(f"從IAD起飛次數最多的航班名是: {most_frequent_carrier}，共 {most_frequent_carrier_count} 次")
if not cancelled_carrier_flights.empty:
    print(f"該航班最常遇到的取消原因是: {most_frequent_cancellation}，出現了 {most_frequent_cancellation_count} 次")
else:
    print(f"該航班從未被取消")

從IAD起飛次數最多的航班名是: AA，共 654 次
該航班最常遇到的取消原因是: B，出現了 289 次


In [16]:
affected_flights = merged_data[merged_data['Description'].isin(['National Air System', 'Security'])]
affected_carriers = affected_flights['CARRIER_NAME'].unique()
all_carriers = merged_data['CARRIER_NAME'].unique()
unaffected_carriers = [carrier for carrier in all_carriers if carrier not in affected_carriers]
carrier_cancellation_counts = affected_flights['CARRIER_NAME'].value_counts()
print("未受到 'National Air System' 和 'Security' 取消航班的航空公司:")
print(unaffected_carriers)
print("\n曾經因 'National Air System' 和 'Security' 取消航班的航空公司以及取消次數:")
print(carrier_cancellation_counts)

未受到 'National Air System' 和 'Security' 取消航班的航空公司:
['AnyCompany Red Airlines']

曾經因 'National Air System' 和 'Security' 取消航班的航空公司以及取消次數:
CARRIER_NAME
AnyCompany Yellow Airlines      3848
AnyCompany Sand Airlines        3617
AnyCompany Blue Airlines        3378
AnyCompany Eclipse Airlines     2747
AnyCompany Green Airlines       2009
AnyCompany Carrot Airlines      1728
AnyCompany Apricot Airlines     1592
AnyCompany Denim Airlines       1520
AnyCompany Pink Airlines        1394
AnyCompany Maroon Airlines       980
AnyCompany Violet Airlines       787
AnyCompany Flint Airlines        708
AnyCompany Harp Airlines         620
AnyCompany Lavender Airlines     198
AnyCompany Indigo Airlines       129
AnyCompany Orange Airlines        85
Name: count, dtype: int64
